
Cyoda Client Demo

Welcome to the Cyoda Client Demo! This notebook demonstrates how to connect and interact with the Cyoda API. Follow the steps below to get started.

## Prerequisites

Before running the cells, ensure you have the following:
- Cyoda API credentials (API key, secret, etc.)
- Necessary Python packages installed

## Steps

1. **Setup**: Import required libraries and set up the environment.
2. **Authentication**: Authenticate with the Cyoda API.
3. **Basic Operations**: Perform basic operations using the API.
4. **Advanced Features**: Explore advanced features and functionalities.

Let's get started!


In [ ]:
pip install -r ../requirements.txt

In [ ]:
# Setup environment variables
import os

API_KEY = os.environ["CYODA_API_KEY"]
API_SECRET = os.environ["CYODA_API_SECRET"]
API_URL = os.environ["CYODA_API_URL"]+"/api"
GRPC_ADDRESS = os.environ["GRPC_ADDRESS"]
WORK_DIR = os.environ["WORK_DIR"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
WORK_DIR = os.environ["WORK_DIR"]
TOKEN = ""
print(API_URL)
print(GRPC_ADDRESS)

In [ ]:
ENTITY_CLASS_NAME = "com.cyoda.tdb.model.treenode.TreeNodeEntity"
ENTITY_NAME="prizes"
AGG_ENTITY_NAME=ENTITY_NAME+"_agg"
MODEL_VERSION="1001"

In [ ]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
# Authenticate with the Cyoda API
import requests
import json

api_url = API_URL + "/auth/login"
headers = {"Content-Type": "application/json", "X-Requested-With": "XMLHttpRequest"}
auth_data = {"username": API_KEY, "password": API_SECRET}
logger.info(api_url)
response = requests.post(api_url, headers=headers, data=json.dumps(auth_data))
if response.status_code == 200:
    logger.info("Authentication successful!")
    TOKEN = response.json().get("token")
else:
    logger.info("Authentication failed. Please check your API credentials.")

In [ ]:
def send_get_request(path):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.get(url, headers=headers)
    return response

In [ ]:
def send_post_request(path, data):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.post(url, headers=headers, data=data)
    return response

In [ ]:
def send_put_request(path, data, timeout):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.put(url, headers=headers, data=data, timeout=timeout)
    return response


In [ ]:
def send_delete_request(path):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.delete(url, headers=headers)
    return response

In [ ]:
def delete_entity_data(entity_name, version):
    path = f"entity/TREE/{entity_name}/{version}"
    response = send_delete_request(path=path)
    logger.info(response)
    return response

In [ ]:
response = delete_entity_data(ENTITY_NAME, MODEL_VERSION)
logger.info(response)

response = delete_entity_data(AGG_ENTITY_NAME, MODEL_VERSION)
logger.info(response)

In [ ]:
def delete_entity_schema(entity_name, version):
    path = f"treeNode/model/{entity_name}/{version}"
    response = send_delete_request(path=path)
    logger.info(response)
    return response

In [ ]:
response = delete_entity_schema(ENTITY_NAME, MODEL_VERSION)
logger.info(response)

response = delete_entity_schema(AGG_ENTITY_NAME, MODEL_VERSION)
logger.info(response)

In [ ]:
def save_entity_schema(entity_name, version, data):
    path = f"treeNode/model/import/JSON/SAMPLE_DATA/{entity_name}/{version}"
    response = send_post_request(path=path, data=data)
    logger.info(response)
    return response

In [ ]:
def test_save_schema(entity_name, file_path):
    data = ''
    try:
        with open(file_path, 'r') as file:
            data = file.read()
            print(data)
    except Exception as e:
        logger.error(f"Failed to read JSON file: {e}")
        return
    response = save_entity_schema(
        entity_name=entity_name, version=MODEL_VERSION, data=data
    )
    logger.info(response)
    assert (
        response.status_code == 200
    ), f"Expected 200, got {response.status_code}"

file_path_base = f"{WORK_DIR}/config-generation/prizes_schema.json"
test_save_schema(ENTITY_NAME, file_path_base)

file_path_agg = f"{WORK_DIR}/config-generation/prizes_agg_schema.json"
test_save_schema(AGG_ENTITY_NAME, file_path_agg)

In [ ]:
def lock_entity_schema(entity_name, version, data):
    path = f"treeNode/model/{entity_name}/{version}/lock"
    response = send_put_request(path=path, data=data, timeout=None)
    logger.info(response)
    return response

In [ ]:
def test_lock_schema(entity_name):
    employees_response = lock_entity_schema(entity_name=entity_name, version=MODEL_VERSION, data=None)
    logger.info(employees_response)
    assert (
        employees_response.status_code == 200
    ), f"Expected 200, got {employees_response.status_code}"


test_lock_schema(ENTITY_NAME)
test_lock_schema(AGG_ENTITY_NAME)

In [ ]:
from typing import List

def save_new_entity(entity_name, version, data):
    path = f"entity/new/JSON/TREE/{entity_name}/{version}"
    response = send_post_request(path=path, data=data)
    # Save entities ids for later use in the tests
    if response.status_code == 200:
        response_json = response.json()
        print(response_json)
    return response

In [ ]:
def test_save_new_entity(entity_name, data):
    employees_response = save_new_entity(
        entity_name=entity_name, version=MODEL_VERSION, data=data
    )
    #logger.info(employees_response.json())
    assert (
        employees_response.status_code == 200
    ), f"Expected 200, got {employees_response.status_code}"

In [ ]:
def test_save_new_entity_from_file(entity_name, file_path):
    
    data = ''
    try:
        with open(file_path, 'r') as file:
            data = file.read()
            print(data)
    except Exception as e:
        logger.error(f"Failed to read JSON file: {e}")
        return
    employees_response = save_new_entity(
        entity_name=entity_name, version=MODEL_VERSION, data=data
    )
    #logger.info(employees_response.json())
    assert (
        employees_response.status_code == 200
    ), f"Expected 200, got {employees_response.status_code}"

#file_path_base = f"{WORK_DIR}/config-generation/prizes_entities.json"
#test_save_new_entity(ENTITY_NAME, file_path_base)

In [ ]:
%%script echo skipping
def get_entity_current_state(entityId):
    
    path = f"platform-api/entity-info/fetch/lazy?entityClass={ENTITY_CLASS_NAME}&entityId={entityId}&columnPath=state"
    response = send_get_request(path=path)
    logger.info(response)
    return response
get_entity_current_state('a057d654-1e01-11b2-89dd-16bcbffd08fd')

In [ ]:
def get_entities(model, version):
    
    path = f"entity/TREE/{model}/{version}"
    response = send_get_request(path=path)
    print(response.json())
    return response
get_entities(ENTITY_NAME, MODEL_VERSION)
get_entities(AGG_ENTITY_NAME, MODEL_VERSION)

...

In [ ]:
# Step 1: Install gRPC and tools
!pip install grpcio grpcio-tools

# Step 2: Compile proto files
!python -m grpc_tools.protoc -I. --python_out=. --pyi_out=. --grpc_python_out=. cyoda-cloud-api.proto

!python -m grpc_tools.protoc -I. --python_out=. --pyi_out=. --grpc_python_out=. cloudevents.proto

In [ ]:

from enum import Enum

class CloudEventType(str, Enum):
    BASE_EVENT = "BaseEvent"
    CALCULATION_MEMBER_JOIN_EVENT = "CalculationMemberJoinEvent"
    CALCULATION_MEMBER_GREET_EVENT = "CalculationMemberGreetEvent"
    ENTITY_PROCESSOR_CALCULATION_REQUEST = "EntityProcessorCalculationRequest"
    ENTITY_PROCESSOR_CALCULATION_RESPONSE = "EntityProcessorCalculationResponse"   

In [ ]:
pip install pandasai

In [ ]:
import json
import pandas as pd
from pandasai import SmartDataframe
from pandasai.llm import OpenAI

def aggregate_prizes(prizes) -> str:
    
    records = []
    for prize in prizes:
        year = prize['year']
        category = prize['category']
        for laureate in prize['laureates']:
            record = {
                "year": year,
                "category": category,
                "id": laureate.get('id', ''),
                "firstname": laureate.get('firstname', ''),
                "surname": laureate.get('surname', ''),
                "motivation": laureate.get('motivation', ''),
                "share": int(laureate.get('share', 0))
            }
            records.append(record)

    df = pd.DataFrame(records)
    llm = OpenAI(api_token=OPENAI_API_KEY)
    sdf = SmartDataframe(df, config={"llm": llm})

    response = sdf.chat('What are top catagories? Produce a report with a chart')
    print(response)

    # Total number of prizes
    total_prizes = len(prizes)

    # Total number of laureates
    total_laureates = df['id'].nunique()

    # Number of prizes per year
    prizes_per_year = df.groupby('year')['category'].nunique().to_dict()

    # Number of prizes per category
    prizes_per_category = df.groupby('category')['year'].nunique().to_dict()

    # Number of laureates per year
    laureates_per_year = df.groupby('year')['id'].nunique().to_dict()

    # Number of laureates per category
    laureates_per_category = df.groupby('category')['id'].nunique().to_dict()

    # Average share per laureate
    average_share_per_laureate = int(df['share'].mean())

    # Total share per prize
    total_share_per_prize = df.groupby(['year', 'category'])['share'].sum().to_dict()

    # Construct the JSON object
    metrics = [
        {
            "metric": "Total number of prizes",
            "value": total_prizes
        },
        {
            "metric": "Total number of laureates",
            "value": total_laureates
        },
        {
            "metric": "Number of prizes per year",
            "value": prizes_per_year
        },
        {
            "metric": "Number of prizes per category",
            "value": prizes_per_category
        },
        {
            "metric": "Number of laureates per year",
            "value": laureates_per_year
        },
        {
            "metric": "Number of laureates per category",
            "value": laureates_per_category
        },
        {
            "metric": "Average share per laureate",
            "value": average_share_per_laureate
        },
        {
            "metric": "Total share per prize",
            "value": total_share_per_prize
        }
    ]

    result = {
        "related_entity_name": "prizes",
        "related_entity_model_version": "10",
        "related_entity_id": "462bad7a-43bf-11b2-9b19-621ba7a93ed7",
        "metrics": metrics
    }

    # Function to convert the tuple keys in the nested dictionary to string keys
    def convert_keys_to_strings(d):
        if isinstance(d, dict):
            return {str(k): convert_keys_to_strings(v) for k, v in d.items()}
        elif isinstance(d, list):
            return [convert_keys_to_strings(i) for i in d]
        else:
            return d

    # Convert the tuple keys in the nested dictionary
    converted_data = convert_keys_to_strings(result)

    # Convert the modified object to JSON
    prizes_data = json.dumps(converted_data, indent=2)
    return prizes_data

In [ ]:
%%script echo skipping
file_path = f"{WORK_DIR}/config-generation/prizes_entities.json"
data = ''
try:
    with open(file_path, 'r') as file:
        data = json.load(file)
        print(data)
except Exception as e:
    logger.error(f"Failed to read JSON file: {e}")

# Flatten the data
prizes = data['data']['prizes']
prizes_agg_result = aggregate_prizes(prizes);
print(prizes_agg_result)

In [ ]:
import grpc
import json
import asyncio
import cloudevents_pb2 as cloudevents_pb2
import cloudevents_pb2_grpc as cloudevents_pb2_grpc
import cyoda_cloud_api_pb2 as cyoda_cloud_api_pb2
import cyoda_cloud_api_pb2_grpc as cyoda_cloud_api_pb2_grpc

def save_prizes_agg_entity(data):
    prizes = data['payload']['data']['data'].get('prizes', [])
    prizes_agg_result = aggregate_prizes(prizes)
    test_save_new_entity(AGG_ENTITY_NAME, prizes_agg_result)

def create_cloud_event(event_id, source, event_type, data) -> cloudevents_pb2.CloudEvent:
    return cloudevents_pb2.CloudEvent(
        id=event_id,
        source=source,
        spec_version="1.0",
        type=event_type,
        text_data=json.dumps(data)
    )

def get_notification(data) -> cloudevents_pb2.CloudEvent:
    return create_cloud_event(
        event_id="8f54729e-994d-4035-8bf9-e7cfe847e2cd",
        source="SimpleSample",
        event_type="EntityProcessorCalculationResponse",
        data={
            "requestId": data['requestId'],
            "entityId": data['entityId'],
            "owner": "PLAY",
            "payload": data['payload'],
            "success": True
        }
    )
    

async def event_producer(queue):
    cloud_event = create_cloud_event(
        event_id="9ba80b3e-e856-4bdb-984b-7523a458101b",
        source="SimpleSample",
        event_type="CalculationMemberJoinEvent",
        data={"owner": "PLAY", "tags": ["prizes"]}
    )

    await queue.put(cloud_event)
    await asyncio.sleep(10)
    #Publish event by saving prizes entity
    test_save_new_entity_from_file(ENTITY_NAME, f"{WORK_DIR}/config-generation/prizes_entities.json")
    await asyncio.sleep(25)
    print("Closing the connection")
    await queue.put(None)
    await asyncio.sleep(3)
    raise asyncio.TimeoutError("Operation timed out!")

async def event_consumer(queue):
    async with grpc.aio.secure_channel(GRPC_ADDRESS, grpc.ssl_channel_credentials()) as channel:
        stub = cyoda_cloud_api_pb2_grpc.CloudEventsServiceStub(channel)

        async def generate_events():
            while True:
                event = await queue.get()
                if event is None:
                    break
                yield event
                queue.task_done()

        async for response in stub.startStreaming(generate_events()):
            print(response)
            data = json.loads(response.text_data)
            if 'processorName' in data and data['processorName'] == 'ai_transform_entity':
                save_prizes_agg_entity(data);
                cloud_event = get_notification(data)
                await queue.put(cloud_event)

async def main():
    queue = asyncio.Queue()
    producer = event_producer(queue)
    consumer = event_consumer(queue)

    await asyncio.gather(producer, consumer)


await main()

In [ ]:
get_entities(ENTITY_NAME, MODEL_VERSION)

In [ ]:
get_entities(AGG_ENTITY_NAME, MODEL_VERSION)

In [ ]:
response = delete_entity_data(ENTITY_NAME, MODEL_VERSION)
logger.info(response)
response = delete_entity_data(AGG_ENTITY_NAME, MODEL_VERSION)
logger.info(response)